In [1]:
import calendar
import pandas as pd
import panel as pn
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import show
pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 2, 12)

### Tables in the process

In [2]:
cols = 'trade name qty price active reason market'.split()
colt = 'trans name spd reason qty target current percent active'.split()

In [3]:
format_dict = {
    'qty':'{:,}','available_qty':'{:,}',
    'price':'{:.2f}','cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}','target':'{:.2f}','current':'{:.2f}',
    'beta':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}','volume':'{:,.2f}',
    'percent':'{:,.2f}'
}

In [4]:
sql = '''
SELECT *
FROM orders
ORDER BY id DESC
LIMIT 1'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

,id,trade,name,qty,price,active,reason,market
0,30,S,WHART,"10,000",12.80,1,DOS,SET


In [5]:
names = orders['name']
name = names.to_string(index=False)

sql = """
SELECT * FROM stocks WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * FROM stocks WHERE name = 'WHART'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,687,WHART,14.08,10.68,I,0.00,12.80,14.07,0.38,12.50,"10,000",-5,4,"10,000",0,0,DOS,SET


### Create orders from stocks after adjust stocks in port_lite from consensus

In [6]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

31

In [7]:
sql = '''
INSERT INTO orders
SELECT id, 'B', name, qty, buy_target, 0, reason, market
FROM stocks
WHERE status IN ('B','O')
ORDER BY name'''
rp = conlite.execute(sql)
rp.rowcount

15

In [8]:
sql = '''
INSERT INTO orders
SELECT id,'S',name, qty, sell_target, 0, reason, market
FROM stocks
WHERE status IN ('I','S')'''
rp = conlite.execute(sql)
rp.rowcount

15

In [9]:
sql = '''
SELECT *
FROM orders
ORDER BY trade, name'''
df_orders = pd.read_sql(sql, conlite)
df_orders.shape[0]

30

In [10]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### After call Ord-log.bat

In [15]:
def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'

In [21]:
df = pd.read_csv('../data/orders-log.csv')
#df.rename(columns = {'col1': 'name', 'col2': 'spd','col3': 'reason', 'col4': 'market',
#                     'col5': 'qty','col6': 'target','col7': 'current', 'col8': 'chg',
#                     'col9': 'percent', 'col10': 'active'}, inplace = True)
df['trans'] = df.apply(lambda row: categorise(row), axis=1)

df_tab = pn.widgets.Tabulator(df, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trade    name  ..., width=800)

In [22]:
df[colt]

,trans,name,spd,reason,qty,target,current,percent,active
0,Buy,ASP,-5,6L,30000,3.72,3.82,0.00%,0
1,Buy,BGC,-5,DOS,10000,10.00,10.50,0.00%,0
2,Buy,CPNCG,-1,DOS,10000,12.60,12.70,0.00%,0
3,Buy,KKP,-11,3L,3000,68.00,70.75,-0.35%,0
4,Buy,MEGA,-5,1L,4500,43.00,44.25,-2.21%,0
5,Buy,NOBLE,-9,RD15%,21000,5.85,6.30,+0.80%,0
6,Buy,RATCH,-13,RD05%,3000,42.75,46.00,0.00%,0
7,Buy,ROJNA,-4,6L,18000,6.30,6.50,-1.52%,0
8,Buy,SCC,-13,RD05pct,300,374.00,387.00,-1.02%,0
9,Buy,SINGER,-3,3L,4500,42.50,43.25,-1.14%,0


In [14]:
format_dict = {
    'qty':'{:,}','available_qty':'{:,}','spd':'{:,}',
    'price':'{:.2f}','cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}','target':'{:.2f}',
    'current':'{:.2f}','beta':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}','volume':'{:,.2f}',
    'percent':'{:,.2f}'
}